In [67]:
from skimage.io import collection, imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
import glob

from datetime import datetime
import nltk

# How is the public opinion about a company correlated to it's market value?

A company's market value is variable and depends on a lot of factors. The price is a reflection of the company's perceived value – what the public is willing to pay for a piece of the company. It can and will rise and fall, based on a variety of factors in the global landscape and within the company itself. One of which is becoming more influential than ever - the public's opinion on social media.

To analyze this correlation we'll look at two datasets. The first one contains over 3 million unique tweets with their information such as tweet id, author of the tweet, post date, the text body of the tweet, and the number of comments, likes, and retweets of tweets matched with the related company.

The second one will just have daily stock price records (from the Forbes2000) for us to make a reference with.

### 1. Data Acquisition 

So first let's read the tweets dataset into pandas and inspect a small sample from the two dataframes.

In [68]:
tweets = pd.read_csv('./top-companies-tweets/Tweet.csv')
tweets.sample(5)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
3286661,1126863635341385728,passthebeano,1557500201,Not yet. Looks like investors are going to get...,0,0,3
1427025,790915214321459200,App_sw_,1477403856,Mastercard is bringing its Masterpass payment ...,0,0,0
2549215,1019926591919542272,TheStreet,1532004423,"$GOOGL's loss, $SSNLF's win? https://buff.ly/2...",2,4,4
1579261,817083269153832961,computer_hware,1483642806,#Apple App Store sets new sales record on New ...,0,0,0
3614936,1191437114937724928,bvisse,1572895718,#tslaq seems to have had a chance last week to...,0,0,1


In [69]:
tweets_company = pd.read_csv('./top-companies-tweets/Company_Tweet.csv')
tweets_company.sample(5)

,tweet_id,ticker_symbol
3087387,889534152088969217,MSFT
4131808,1131415166959140864,TSLA
3472560,832323193239117827,TSLA
2119366,1149465145413734401,GOOGL
1043833,979858566927429633,AAPL


So next up let's read the datasets for each of the stocks, which we are monitoring. We will save them in a dictionary with the key, being the company's tick name and the value - it's stock prices over time dataset.

In [70]:
stocks_df = {}
for name in glob.glob('./stocks/*'):
    stocks_df[name.split('\\')[-1].split('.')[0]] = pd.read_csv(name)
stocks = pd.concat(stocks_df)
stocks.sample(10)

Date          Low         Open     Volume         High  \
AMZN 5629  27-09-2019  1713.819946  1748.000000    3907200  1749.119995   
     1628  04-11-2003    56.060001    56.700001    7009500    57.490002   
     5287  18-05-2018  1572.099976  1581.329956    2642600  1583.589966   
TSLA 1218  01-05-2015    44.082001    45.987999   26408500    46.354000   
AAPL 4140  29-04-1997     0.156250     0.160714   51755200     0.160714   
     2537  26-12-1990     0.383929     0.392857  103073600     0.395089   
TSLA 2169  11-02-2019    62.099998    62.320000   35648500    63.720001   
     323   07-10-2011     5.210000     5.396000    6558000     5.520000   
AAPL 1068  05-03-1985     0.115513     0.115513  130771200     0.115513   
     6371  13-03-2006     2.313929     2.323214  861187600     2.367143   

                 Close  Adjusted Close  
AMZN 5629  1725.449951     1725.449951  
     1628    56.860001       56.860001  
     5287  1574.369995     1574.369995  
TSLA 1218    45.206001       45.206001  
AAPL 4140     0.157924        0.135799  
     2537     0.390625        0.317737  
TSLA 2169    62.568001       62.568001  
     323      5.398000        5.398000  
AAPL 1068     0.115513        0.090676  
     6371     2.345714        2.017086

### 2. Data Tidying and Cleaning

First let's combine the two tables from the twitter dataset, convert the dates to a datetime object and rename the column.

In [71]:
tweets = pd.merge(tweets, tweets_company, on = "tweet_id")

In [72]:
tweets["date"] = pd.to_datetime(tweets.post_date, unit='s')
tweets = tweets.drop(columns="post_date")
tweets.sample(5)


,tweet_id,writer,body,comment_num,retweet_num,like_num,ticker_symbol,date
3015445,1026911680213729280,AGORACOM,Given the fact middle east is charging fast in...,0,0,0,TSLA,2018-08-07 19:23:18
4276749,1202304304096403458,papaspoppinpeny,$NRPI and Luxium: The Revolutionary Unique Met...,0,0,0,AMZN,2019-12-04 19:10:58
1865121,827159021744431104,strader55,uranium-pinto-beans: $AAPL eanrings worse than...,0,0,0,AAPL,2017-02-02 14:17:33
1733662,806157630251106304,shonkori,the lure of the cloud may be enough to draw cu...,1,0,0,AMZN,2016-12-06 15:25:31
2921922,1015250170399817728,rimisback,BlackBerry the phone in the market for right r...,0,1,3,GOOGL,2018-07-06 15:04:37


We will see what timeframe does our dataset cover, by getting the data of the earliest and latest tweets.

In [73]:
tweets.post_date.min(), tweets.post_date.max()

AttributeError: 'DataFrame' object has no attribute 'post_date'

So it has data from 01.01.2015 to 31.12.2019, so basically from 2015 to the beginning of 2020. Knowing this we can filter out the stock prices to be only in this period of time. But first we have to covert the "Date" column to datetime.

In [ ]:
def string_to_date(date_string):
    return datetime.strptime(date_string, "%d-%m-%Y")
stocks.Date = pd.to_datetime(stocks.Date.apply(string_to_date))

In [ ]:
stocks = stocks[(stocks.Date >= '01-01-2015') & (stocks.Date < '01-01-2020')]
stocks.sample(10)

Date         Low        Open     Volume        High  \
AAPL 9177 2017-05-04   36.452499   36.630001   93487600   36.785000   
     8617 2015-02-12   31.392500   31.514999  297898000   31.870001   
     9065 2016-11-21   27.502501   27.530001  117058400   27.997499   
GOOG 2665 2015-03-23  554.308167  558.895569    1643801  560.820251   
MSFT 7774 2017-01-12   61.950001   63.060001   20968200   63.400002   
AAPL 9086 2016-12-21   29.195000   29.200001   95132800   29.350000   
MSFT 8363 2019-05-17  127.919998  128.309998   25770500  130.460007   
GOOG 3131 2017-01-26  827.010010  837.809998    2973900  838.000000   
AMZN 5113 2017-09-08  963.469971  979.099976    2605300  979.880005   
     4547 2015-06-11  431.470001  432.290009    2922500  438.890015   

                Close  Adjusted Close  
AAPL 9177   36.632500       34.733509  
     8617   31.615000       28.821255  
     9065   27.932501       26.370180  
GOOG 2665  557.279968      557.279968  
MSFT 7774   62.610001       58.431557  
AAPL 9086   29.264999       27.628145  
MSFT 8363  128.070007      125.299995  
GOOG 3131  832.150024      832.150024  
AMZN 5113  965.900024      965.900024  
     4547  432.970001      432.970001

As we can see we don't have data for every day, because the stock market funcitons only on workdays, unlike twitter.  We will find a way to work around this later.

Now lets strip the data down to just one column - the value, which we will calculate by getting the mean of the Open and Close prices. 

In [74]:
stocks["Value"] = (stocks.Open + stocks.Close) / 2
stocks = stocks.drop(columns=['Low', 'Open', 'High', 'Close', 'Adjusted Close'])
stocks.sample(10)

Date     Volume       Value
AAPL 9626  15-02-2019   98507200   42.708750
MSFT 8274  09-01-2019   32280800  104.064999
GOOG 1696  13-05-2011    4233208  265.046539
AAPL 1449  08-09-1986  126201600    0.155692
TSLA 2369  25-11-2019   61697500   68.065998
MSFT 1880  18-08-1993   66012800    2.421875
AMZN 1649  04-12-2003   10216200   51.625000
     4523  07-05-2015    2766100  425.514999
AAPL 5576  14-01-2003  186860800    0.261607
     4594  17-02-1999  296060800    0.335379

It is a little inconvenient to have the stock name as an index instead of it being a regular column. We will fix that and also change the column names to match the twitter dataset.

In [76]:
stocks = stocks.reset_index(level=0)

In [77]:
stocks.columns = ["ticker_symbol", "date", "volume", "value"]
stocks.sample(10)
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32271 entries, 0 to 2782
Data columns (total 4 columns):
ticker_symbol    32271 non-null object
date             32271 non-null object
volume           32271 non-null int64
value            32271 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.2+ MB


 So next up lets take a look at the datatypes and null values for the twitter dataset.

In [78]:
tweets.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
tweet_id         4336445 non-null int64
writer           4280526 non-null object
body             4336445 non-null object
comment_num      4336445 non-null int64
retweet_num      4336445 non-null int64
like_num         4336445 non-null int64
ticker_symbol    4336445 non-null object
date             4336445 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 297.8+ MB


Everything looks good, except the ticker_symbol which should be a category. The code below fixes that.

In [88]:
tweets.ticker_symbol = tweets.ticker_symbol.astype('category')
tweets.head(7)

,tweet_id,writer,body,comment_num,retweet_num,like_num,ticker_symbol,date
0,550441509175443456,VisualStockRSRC,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,2015-01-01 00:00:57
1,550441672312512512,KeralaGuy77,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,2015-01-01 00:01:36
2,550441732014223360,DozenStocks,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN,2015-01-01 00:01:50
3,550442977802207232,ShowDreamCar,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA,2015-01-01 00:06:47
4,550443807834402816,i_Know_First,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05
5,550443807834402816,i_Know_First,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,TSLA,2015-01-01 00:10:05
6,550443808606126081,aaplstocknews,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05


As we can see there seem to be a lot of duplicate bodies in our dataset. We want to remove them and this is exactly what the following code does. 

In [94]:
tweets = tweets.drop_duplicates(subset=["body"])

3326194

### 3. Text Preparation and Exploration

Before we start working with the text, we have to prepare it and take a quick look at some statistics about it.

In [86]:
tweets.body.duplicated().value_counts()

False    3326194
True     1010251
Name: body, dtype: int64